In [1]:
import pandas as pd
import numpy as np


In [3]:
df=pd.read_csv("data.csv")

In [5]:
df.head()

,Sentence,Sentiment
0,The GeoSolutions technology will leverage Bene...,positive
1,"$ESI on lows, down $1.50 to $2.50 BK a real po...",negative
2,"For the last quarter of 2010 , Componenta 's n...",positive
3,According to the Finnish-Russian Chamber of Co...,neutral
4,The Swedish buyout firm has sold its remaining...,neutral


In [21]:
para=df['Sentence']

In [23]:
import nltk
from nltk.tokenize import sent_tokenize

In [27]:
df.shape

(5842, 2)

In [29]:
df.isnull().sum()

Sentence     0
Sentiment    0
dtype: int64

In [31]:
# preprocessing and cleaning

In [45]:
df['Sentiment'].unique()

array(['positive', 'negative', 'neutral'], dtype=object)

In [57]:
df['Sentiment'] = df['Sentiment'].replace({
    'positive': 1,
    'neutral': 1,
    'negative': 0  
})

In [59]:
df.head()

,Sentence,Sentiment
0,The GeoSolutions technology will leverage Bene...,1
1,"$ESI on lows, down $1.50 to $2.50 BK a real po...",0
2,"For the last quarter of 2010 , Componenta 's n...",1
3,According to the Finnish-Russian Chamber of Co...,1
4,The Swedish buyout firm has sold its remaining...,1


In [61]:
## lower all the cases
df['Sentence']=df['Sentence'].str.lower()

In [67]:
df['Sentence'].value_counts()

managing director 's comments : `` net sales for the first quarter were notably lower than a year before , especially in finland , russia and the baltic countries .                                                          2
scanfil , a systems supplier and contract manufacturer to the communications sector , will give notice to 20 productional employees , and temporarily lay off 15 employees at the company 's plant in sievi , in finland .    2
profit before taxes decreased to eur 31.6 mn from eur 50.0 mn the year before .                                                                                                                                               2
pretax loss totalled eur 49.9 mn , compared to a loss of eur 15.4 mn in the corresponding period in 2008 .                                                                                                                    2
operating loss of the pulp & paper machinery unit was over eur 3mn in september 2007 - august 2008 , com

In [71]:
import re
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\being\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [69]:
from bs4 import BeautifulSoup

In [73]:
## Removing special characters
df['Sentence']=df['Sentence'].apply(lambda x:re.sub('[^a-z A-z 0-9-]+', '',x))
## Remove the stopswords
df['Sentence']=df['Sentence'].apply(lambda x:" ".join([y for y in x.split() if y not in stopwords.words('english')]))
## Remove url 
df['Sentence']=df['Sentence'].apply(lambda x: re.sub(r'(http|https|ftp|ssh)://([\w_-]+(?:(?:\.[\w_-]+)+))([\w.,@?^=%&:/~+#-]*[\w@?^=%&/~+#-])?', '' , str(x)))
## Remove html tags
df['Sentence']=df['Sentence'].apply(lambda x: BeautifulSoup(x, 'lxml').get_text())
## Remove any additional spaces
df['Sentence']=df['Sentence'].apply(lambda x: " ".join(x.split()))

In [75]:
df.head()

,Sentence,Sentiment
0,geosolutions technology leverage benefon gps s...,1
1,esi lows 150 250 bk real possibility,0
2,last quarter 2010 componenta net sales doubled...,1
3,according finnish-russian chamber commerce maj...,1
4,swedish buyout firm sold remaining 224 percent...,1


In [77]:
# Lemmatizer
from nltk.stem import WordNetLemmatizer
lemmatizer=WordNetLemmatizer()

In [79]:
def lemmatize_words(text):
    return " ".join([lemmatizer.lemmatize(word) for word in text.split()])

In [83]:
df['Sentence']=df['Sentence'].apply(lambda x:lemmatize_words(x))

In [85]:
df.head()

,Sentence,Sentiment
0,geosolutions technology leverage benefon gps s...,1
1,esi low 150 250 bk real possibility,0
2,last quarter 2010 componenta net sale doubled ...,1
3,according finnish-russian chamber commerce maj...,1
4,swedish buyout firm sold remaining 224 percent...,1


In [89]:
# Train test split
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(df['Sentence'],df['Sentiment'],test_size=0.20)

In [91]:
from sklearn.feature_extraction.text import CountVectorizer
bow=CountVectorizer()
X_train_bow=bow.fit_transform(X_train).toarray()
X_test_bow=bow.transform(X_test).toarray()

In [93]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf=CountVectorizer()
X_train_tfidf=tfidf.fit_transform(X_train).toarray()
X_test_tfidf=tfidf.transform(X_test).toarray()

In [95]:
from sklearn.naive_bayes import GaussianNB
nb_model_bow=GaussianNB().fit(X_train_bow,y_train)
nb_model_tfidf=GaussianNB().fit(X_train_tfidf,y_train)

In [97]:
from sklearn.metrics import confusion_matrix,accuracy_score,classification_report

In [99]:
y_pred_bow=nb_model_bow.predict(X_test_bow)
y_pred_tfidf=nb_model_tfidf.predict(X_test_tfidf)

In [101]:
confusion_matrix(y_test,y_pred_bow)

array([[ 57, 108],
       [222, 782]], dtype=int64)

In [103]:
print("BOW accuracy: ",accuracy_score(y_test,y_pred_bow))

BOW accuracy:  0.7177074422583405


In [105]:
confusion_matrix(y_test,y_pred_tfidf)

array([[ 57, 108],
       [222, 782]], dtype=int64)

In [107]:
print("TFIDF accuracy: ",accuracy_score(y_test,y_pred_tfidf))

TFIDF accuracy:  0.7177074422583405
